In [ ]:
# import the necessary packages
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense
from keras.models import Sequential
import tensorflow as tf

import glob, os, random, math
import numpy as np

In [ ]:
# training, validation and testing directories

train_dir='./train_valid_test/train/'
valid_dir='./train_valid_test/valid/'
test_dir='./train_valid_test/test/'

train_list=glob.glob(os.path.join(train_dir,'*/*.jpg'))
valid_list = glob.glob(os.path.join(valid_dir, '*/*.jpg'))
test_list = glob.glob(os.path.join(test_dir, '*/*.jpg'))

print("No of images in ")
print("\tTraining set : ",len(train_list))
print("\tValidation set : ",len(valid_list))
print("\tTest set : ",len(test_list))
print("\nTotal : ",len(train_list)+len(valid_list)+len(test_list))

In [ ]:
# data visualization
fig=plt.figure(figsize=(8, 8))
columns = 4
rows = 4
for i in range(1, columns*rows +1):
    img = load_img(random.choice(train_list))
    fig.add_subplot(rows, columns, i)
    plt.tight_layout(pad=2.0)
    plt.imshow(img)
plt.show()

In [ ]:
# data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

print("Train : ",end="")
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    seed=0
)

print("Valid : ",end="")
validation_generator = train_datagen.flow_from_directory(
    valid_dir,
    target_size=(300, 300),
    batch_size=32,
    class_mode='categorical',
    shuffle=False,
    seed=0
)

print("Test : ",end="")
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(300,300),
    batch_size=24,
    class_mode='categorical',
    seed=0,
    shuffle=True
)

labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

print("\nClasses",labels)

In [ ]:
# network
model = Sequential([
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(300, 300, 3)),
    MaxPooling2D(pool_size=2),

    Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),
    
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),

    Flatten(),

    Dense(64, activation='relu'),

    Dense(6, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

model.summary()

In [ ]:
# training and validation
model.fit_generator(
    train_generator,
    validation_data = validation_generator, 
    epochs = 50)

In [ ]:
# testing
test_x, test_y = test_generator.__getitem__(1)

preds = model.predict(test_x)

total = test_generator.batch_size
plt.figure(figsize=(total, total))
correct=0

rows = math.ceil(total/4)

print("Prediction on a single batch\n")
for i in range(total):
    plt.subplot(rows, 4, i+1)
    plt.title('pred:%s / truth:%s' % (labels[np.argmax(preds[i])], labels[np.argmax(test_y[i])]))
    plt.imshow(test_x[i])
    if (labels[np.argmax(preds[i])] == labels[np.argmax(test_y[i])]):
        correct+=1
print(f"{correct} prediction correct out of {total}")
print(f"Accuracy : {(correct/total)*100}")